# Модель определения токсичных комментариев
**Проект №12 Яндекс.Практикум - Data Science**

## Описание проекта

**Исходные данные:**

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию.

В вашем распоряжении набор данных с разметкой о токсичности правок.

**Цель проекта:**

Разработать модель классификации комментариев на позитивные и негативные.

**Условия задачи:**

Значение метрики качества **F1** не меньше **0.75**. 

### Структура проекта
* [1. Загрузка и анализ данных](#start)
* [2. Подготовка данных](#preparation)
* [3. Обучение модели](#model)
* [4. Тестирование](#testing)
* [5. Выводы](#conclusion)

<a id="start"></a>
## 1. Загрузка и анализ данных

#### Импортируем необходимые библиотеки

In [3]:
import re
import nltk
import string
import spacy
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from nltk.corpus import stopwords 
from textblob import TextBlob, Word

In [2]:
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Sergio\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Sergio\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Sergio\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Sergio\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

#### Загрузим данные в DataFrame

In [3]:
dataset = 'toxic_comments.csv'

try:
    data = pd.read_csv(f'../datasets/{dataset}', sep=',')
    print(f'Прочитан файл с данными: "./datasets/{dataset}"')
except:
    try:
        data = pd.read_csv(f'/datasets/{dataset}', sep=',') # yandex.praktikum
        print(f'Прочитан файл с данными: "/datasets/{dataset}"')
    except Exception as err:
        print(repr(err))

Прочитан файл с данными: "./datasets/toxic_comments.csv"


In [4]:
data.head(5)

,text,toxic
0,Explanation\nWhy the edits made under my usern...,0
1,D'aww! He matches this background colour I'm s...,0
2,"Hey man, I'm really not trying to edit war. It...",0
3,"""\nMore\nI can't make any real suggestions on ...",0
4,"You, sir, are my hero. Any chance you remember...",0


In [5]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159571 entries, 0 to 159570
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    159571 non-null  object
 1   toxic   159571 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ MB


#### Описание данных:
* text - текст комментария
* toxic — целевой признак

In [6]:
print(f"Кол-во текстов класса 0: {data['toxic'].value_counts()[0]} ({(data['toxic'].value_counts()[0]/data.shape[0])*100:.2f}%)")
print(f"Кол-во текстов класса 1: {data['toxic'].value_counts()[1]} ({(data['toxic'].value_counts()[1]/data.shape[0])*100:.2f}%)")

Кол-во текстов класса 0: 143346 (89.83%)
Кол-во текстов класса 1: 16225 (10.17%)


#### Выводы:
* отсутствуют пропуски в данных
* классы несбалансированы, учтём это при обучении моделей

<a id="preparation"></a>
## 2. Подготовка данных

Лемматиизируем текст с помощью библиотек **spacy** и **TextBlob**

Реализуем функции лемматизации

In [7]:
# TextBlob lemmatizer
def lemmatize_tblob(text):
    sentence = TextBlob(text)
    tag_dict = {"J": 'a', 
                "N": 'n', 
                "V": 'v', 
                "R": 'r'}
    words_and_tags = [(w, tag_dict.get(pos[0], 'n')) for w, pos in sentence.tags]    
    lemmatized_list = [wd.lemmatize(tag) for wd, tag in words_and_tags]
    return " ".join(lemmatized_list)

In [8]:
# Spacy lemmatizer
def lemmatize_spacy(text, lemmatizer):
    doc = lemmatizer(text)
    lemm_text = " ".join([token.lemma_ for token in doc])
        
    return lemm_text

Функция очистки текста (оставляет только буквы, кавычки и пробелы)

In [9]:
def clear(text):
    cleaned = re.sub(r"[^a-zA-Z\' ]", ' ', text)
    return " ".join(cleaned.split())

Лемматизируем текст

In [10]:
%%time

data['lemm_tblob'] = data['text'].apply(lemmatize_tblob)

Wall time: 22min 16s


In [11]:
%%time

sp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

data['lemm_spacy'] = data['text'].apply(lemmatize_spacy, lemmatizer=sp)

Wall time: 18min 30s


Очистим лемматизированных текст от лишних символов

In [12]:
%%time

data['lemm_tblob'] = data['lemm_tblob'].apply(clear)
data['lemm_spacy'] = data['lemm_spacy'].apply(clear)
data

Wall time: 4.21 s


,text,toxic,lemm_tblob,lemm_spacy
0,Explanation\nWhy the edits made under my usern...,0,Explanation Why the edits make under my userna...,Explanation why the edit make under my usernam...
1,D'aww! He matches this background colour I'm s...,0,D'aww He match this background colour I 'm see...,D'aww he match this background colour I be see...
2,"Hey man, I'm really not trying to edit war. It...",0,Hey man I 'm really not try to edit war It 's ...,hey man I be really not try to edit war it be ...
3,"""\nMore\nI can't make any real suggestions on ...",0,More I ca n't make any real suggestion on impr...,More I ca n't make any real suggestion on impr...
4,"You, sir, are my hero. Any chance you remember...",0,You sir be my hero Any chance you remember wha...,you sir be my hero any chance you remember wha...
...,...,...,...,...
159566,""":::::And for the second time of asking, when ...",0,And for the second time of ask when your view ...,and for the second time of ask when your view ...
159567,You should be ashamed of yourself \n\nThat is ...,0,You should be ashamed of yourself That be a ho...,you should be ashamed of yourself that be a ho...
159568,"Spitzer \n\nUmm, theres no actual article for ...",0,Spitzer Umm theres no actual article for prost...,spitzer Umm there s no actual article for pros...
159569,And it looks like it was actually you who put ...,0,And it look like it be actually you who put on...,and it look like it be actually you who put on...


In [13]:
print(data['lemm_spacy'][4])
print(data['lemm_tblob'][4])
print(data['lemm_spacy'][5])
print(data['lemm_tblob'][5])
print(data['lemm_spacy'][6])
print(data['lemm_tblob'][6])
print(data['lemm_spacy'][9])
print(data['lemm_tblob'][9])
print(data['lemm_spacy'][11])
print(data['lemm_tblob'][11])

you sir be my hero any chance you remember what page that be on
You sir be my hero Any chance you remember what page that 's on
congratulation from I as well use the tool well talk
Congratulations from me a well use the tool well talk
COCKSUCKER before you pis around on my work
COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
alignment on this subject and which be contrary to those of DuLithgow
alignment on this subject and which be contrary to those of DuLithgow
bbq be a man and let discuss it maybe over the phone
bbq be a man and let discuss it maybe over the phone


#### Выводы:
* лемматизировали тексты используя библиотеки **spacy** и **TextBlob**
* для **TextBlob** применили технику POS-tag (part-of-speech), в **spacy** она используется по умолчанию
* значимых отличий в лемматизированных текстах разными библиотеками не замечено
* лемматизация с **spacy** работает незначительно быстрее, чем с **TextBlob**
* сравним на обученных моделях есть ли различия в качестве при использования лемматизированных текстов используя **spacy** и **TextBlob**

<a id="model"></a>
## 3. Обучение модели

Разделим данные на тестовую и тренировочную выборку в пропорции 4:1

In [14]:
target = data['toxic']
# Do not use extra space
corpus_train, corpus_test, target_train, target_test = train_test_split(data['lemm_spacy'].values.astype('U'), target, test_size=0.2, stratify=target, random_state=42)

Создадим матрицу оценки важности слов, рассчитав для каждого слова TF-IDF

In [15]:
stop_words = set(stopwords.words('english'))

In [16]:
count_vect = TfidfVectorizer(stop_words=stop_words)
tf_idf  = count_vect.fit_transform(corpus_train) 
print("Размер массива TF-IDF:", tf_idf.shape)

Размер массива TF-IDF: (127656, 138588)


In [17]:
target = data['toxic']
# Do not use extra space
corpus_train_tblob, corpus_test_tblob, target_train_tblob, target_test_tblob = train_test_split(data['lemm_tblob'].values.astype('U'), target, test_size=0.2, stratify=target, random_state=42)

In [18]:
stop_words = set(stopwords.words('english'))

count_vect_tblob = TfidfVectorizer(stop_words=stop_words)
tf_idf_tblob  = count_vect_tblob.fit_transform(corpus_train) 
print("Размер массива TF-IDF:", tf_idf_tblob.shape)

Размер массива TF-IDF: (127656, 138588)


#### Random Forrest

In [19]:
%%time

# Parameters selected on smaller amount of data 
model = RandomForestClassifier(ccp_alpha=0.005,
                               n_jobs=-1)

parameters = {'max_depth' : [None, 30],
              'class_weight' : [None, {0: 1, 1: 5}, {0: 1, 1: 8}]}

clf = GridSearchCV(model, parameters)
clf.fit(tf_idf, target_train)
model_rfc = clf.best_estimator_
model_rfc

Wall time: 1h 14min


RandomForestClassifier(bootstrap=True, ccp_alpha=0.005,
                       class_weight={0: 1, 1: 8}, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       max_samples=None, min_impurity_decrease=0.0,
                       min_impurity_split=None, min_samples_leaf=1,
                       min_samples_split=2, min_weight_fraction_leaf=0.0,
                       n_estimators=100, n_jobs=-1, oob_score=False,
                       random_state=None, verbose=0, warm_start=False)

Проверим F1 и баланс классов на обучающей выборке 

In [20]:
predicted = model_rfc.predict(tf_idf)
f1 = f1_score(target_train, predicted)
print(f1)

unique, counts = np.unique(predicted, return_counts=True)
print(dict(zip(unique, counts)))

0.08057362507392077
{0: 127108, 1: 548}


#### Logistic Regression

In [21]:
%%time

model = LogisticRegression(solver='lbfgs',
                           n_jobs=-1,
                           random_state=91)

parameters = {'class_weight': [{0: 1, 1: 5}, {0: 1, 1: 7}]}

clf = GridSearchCV(model, parameters)
clf.fit(tf_idf, target_train)
model_logreg = clf.best_estimator_
model_logreg

Wall time: 49.8 s


LogisticRegression(C=1.0, class_weight={0: 1, 1: 5}, dual=False,
                   fit_intercept=True, intercept_scaling=1, l1_ratio=None,
                   max_iter=100, multi_class='auto', n_jobs=-1, penalty='l2',
                   random_state=91, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

Проверим F1 и баланс классов на обучающей выборке 

In [22]:
predicted = model_logreg.predict(tf_idf)
f1 = f1_score(target_train, predicted)
print(f1)

unique, counts = np.unique(predicted, return_counts=True)
print(dict(zip(unique, counts)))

0.8742113422504237
{0: 112899, 1: 14757}


#### Выводы:
* подобраны наилучшие гиперпараметры для **Random Forrest** и для **Logistic Regression**
* на Random Forrest не удалось получить приемлимое значение F1 даже на обучающих данных, при этом при таком большом количестве параметро обучается очень долго
* Logistic Regression немного переобучается

<a id="testing"></a>
## 4. Тестирование

Посчитаем TF-IDF для тестовой выборки

In [23]:
tf_idf_test = count_vect.transform(corpus_test) 
print("Размер массива TF-IDF:", tf_idf_test.shape)

Размер массива TF-IDF: (31915, 138588)


In [24]:
predicted_test = model_rfc.predict(tf_idf_test)
print(f"F1 score Random Forrest: {f1_score(target_test, predicted_test)}")

F1 score Random Forrest: 0.09117647058823529


In [25]:
predicted_test = model_logreg.predict(tf_idf_test)
print(f"F1 score Logistic Regression: {f1_score(target_test, predicted_test)}")

F1 score Logistic Regression: 0.7809917355371901


Сравним есть ли разница по сравнению с лемматизированных текстом с помощью TextBLob

In [26]:
tf_idf_test_tblob = count_vect_tblob.transform(corpus_test_tblob) 
print("Размер массива TF-IDF:", tf_idf_test_tblob.shape)

Размер массива TF-IDF: (31915, 138588)


In [27]:
predicted_test_tblob = model_logreg.predict(tf_idf_test_tblob)
print(f"F1 score Logistic Regression (TextBlob): {f1_score(target_test, predicted_test)}")

F1 score Logistic Regression (TextBlob): 0.7809917355371901


<a id="conclusion"></a>
## 5. Выводы

* лемматизировали текст используя библиотеки **spacy** и **TextBlob**
* сравнили результаты модели классификации на леммах сделанных **spacy** и **TextBlob**, и выявили, что отличий нет
* обучили модели Random Forrest и Logistic Regression
* обучали модели, учитывая дисбаланс классов, и подобрали наилучшие гиперпараметры, использоу Grid Search
* выявили, что для задачи классификации текстов лучше подходит модель Logistic Regression
* на тестовой выборке получили значение F1 = 0.78